In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [2]:
import wandb
project_name = "object_detection"
wandb.init(project=project_name, reinit=True)

wandb: Currently logged in as: jongeson (use `wandb login --relogin` to force relogin)


In [3]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/trash/cascade/cascade_rcnn_r50_fpn_1x_trash.py')

PREFIX = '../../input/data/'


In [4]:
wandb.run.name = 'cascade_rcnn_r50_fpn_1x_trash'
# generted run ID로 하고 싶다면 다음과 같이 쓴다.
# wandb.run.name = wandb.run.id
wandb.run.save()

wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


True

In [12]:
cfg.log_config.keys()

dict_keys(['interval', 'hooks'])

In [5]:
cfg.log_config.hooks = [
	dict(type='TextLoggerHook'),
	dict(
		type='WandbLoggerHook',
		init_kwargs=dict(
				project=project_name,
				name='cascade_rcnn_r50_fpn_1x_trash'
				)
		)
]

In [7]:
# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

# num_workers?
cfg.data.samples_per_gpu = 4

# seed
cfg.seed=2020

# ??
cfg.gpu_ids = [0]

# 
cfg.work_dir = './work_dirs/cascade_rcnn_r50_fpn_1x_trash'

# ??
# cfg.model.roi_head.bbox_head.num_classes = 11

# ??
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

In [8]:
model = build_detector(cfg.model)

2021-05-13 07:54:10,731 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-13 07:54:10,733 - mmdet - INFO - Use load_from_torchvision loader
2021-05-13 07:54:11,023 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



In [9]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.42s)
creating index...
index created!


In [10]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-13 07:54:47,185 - mmdet - INFO - Start running, host: root@a868eb526110, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/cascade_rcnn_r50_fpn_1x_trash
2021-05-13 07:54:47,190 - mmdet - INFO - workflow: [('train', 1)], max: 12 epochs


Done (t=1.16s)
creating index...
index created!


2021-05-13 07:55:21,383 - mmdet - INFO - Epoch [1][50/655]	lr: 1.978e-03, eta: 1:09:32, time: 0.534, data_time: 0.049, memory: 2528, loss_rpn_cls: 0.4936, loss_rpn_bbox: 0.0730, s0.loss_cls: 0.8041, s0.acc: 83.6318, s0.loss_bbox: 0.1635, s1.loss_cls: 0.3322, s1.acc: 86.5410, s1.loss_bbox: 0.0479, s2.loss_cls: 0.1821, s2.acc: 82.2012, s2.loss_bbox: 0.0067, loss: 2.1031, grad_norm: 8.9631
2021-05-13 07:55:47,129 - mmdet - INFO - Epoch [1][100/655]	lr: 3.976e-03, eta: 1:07:50, time: 0.515, data_time: 0.006, memory: 2528, loss_rpn_cls: 0.2219, loss_rpn_bbox: 0.0683, s0.loss_cls: 0.4633, s0.acc: 89.5273, s0.loss_bbox: 0.2447, s1.loss_cls: 0.1343, s1.acc: 95.2520, s1.loss_bbox: 0.0732, s2.loss_cls: 0.0418, s2.acc: 97.5830, s2.loss_bbox: 0.0096, loss: 1.2573, grad_norm: 3.7143
2021-05-13 07:56:12,384 - mmdet - INFO - Epoch [1][150/655]	lr: 5.974e-03, eta: 1:06:34, time: 0.505, data_time: 0.006, memory: 2528, loss_rpn_cls: 0.1925, loss_rpn_bbox: 0.0604, s0.loss_cls: 0.5052, s0.acc: 87.9502, s0

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 14.7 task/s, elapsed: 45s, ETA:     0s

...

Loading and preparing results...
DONE (t=0.53s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.37s).
Accumulating evaluation results...


2021-05-13 09:12:18,117 - mmdet - INFO - Epoch(val) [12][655]	bbox_mAP: 0.1790, bbox_mAP_50: 0.2860, bbox_mAP_75: 0.1870, bbox_mAP_s: 0.0350, bbox_mAP_m: 0.1560, bbox_mAP_l: 0.3610, bbox_mAP_copypaste: 0.179 0.286 0.187 0.035 0.156 0.361


DONE (t=0.68s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.179
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.286
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.187
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.035
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.156
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.361
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.321
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.321
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.321
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.100
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.286
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.601


train/loss_rpn_cls,0.03221
train/loss_rpn_bbox,0.02819
train/s0.loss_cls,0.20829
train/s0.acc,93.31934
train/s0.loss_bbox,0.10313
train/s1.loss_cls,0.09403
train/s1.acc,94.02276
train/s1.loss_bbox,0.09442
train/s2.loss_cls,0.04802
train/s2.acc,94.0192
train/s2.loss_bbox,0.05998


train/loss_rpn_cls,█▆▇▅▄▅▄▃▃▄▃▃▃▃▃▃▂▂▂▃▂▃▃▂▂▃▂▂▁▂▁▁▁▁▁▁▁▁▁▁
train/loss_rpn_bbox,▇▇█▇▅▆▄▅▄▅▄▄▄▄▅▄▃▃▃▃▃▄▄▃▃▄▃▂▂▂▁▂▂▁▂▂▁▂▁▂
train/s0.loss_cls,▇██▇▆▇▆▆▅▅▄▅▅▄▅▅▄▅▄▅▃▄▄▄▃▄▄▂▂▂▂▂▂▂▂▂▁▁▁▁
train/s0.acc,▃▁▁▂▃▁▂▂▂▂▄▂▃▄▃▃▄▃▄▃▅▄▃▄▆▄▄▇▇▆▇▇▆▇▇▇████
train/s0.loss_bbox,▇█▆▆▄▅▅▄▄▄▃▄▃▄▄▄▃▃▃▃▂▃▃▃▂▃▃▂▁▂▁▁▂▁▁▁▁▁▁▁
train/s1.loss_cls,▄▅█▇▇██▇▆▇▅▇▆▅▆▆▅▆▅▆▄▅▅▅▄▅▅▃▂▃▂▂▃▂▂▂▁▁▁▂
train/s1.acc,█▅▃▃▃▁▂▁▂▂▃▁▃▃▂▃▃▂▃▃▄▃▃▃▅▃▃▅▆▅▆▆▅▆▆▆▇▇▇▇
train/s1.loss_bbox,▁▅▆▇▆█▇▇▇▇▆▇▆▆▆▆▅▅▅▅▅▅▆▅▄▅▅▄▃▄▃▃▄▃▃▃▃▃▃▃
train/s2.loss_cls,▁▂▅▆▆███▇▇▆▇▆▆▇▇▆▆▅▇▅▆▆▆▄▆▆▃▃▃▂▃▃▂▂▂▁▂▁▂
train/s2.acc,█▇▅▃▃▁▂▁▂▂▃▁▂▂▁▂▂▂▂▂▃▂▂▂▃▂▂▄▄▄▄▄▃▅▄▄▅▅▅▅
train/s2.loss_bbox,▁▂▅▆▇█▇██▇▇█▇▇██▇▇▇▇▇▇█▇▆▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆
